# Window Operations Exercise

### Exercise

In [1]:
import findspark
# TODO: your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/home/matthew/spark-2.1.0-bin-hadoop2.7')

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
import sys
import random
from apache_log_parser import ApacheAccessLog

conf = (SparkConf().setMaster("local[4]").setAppName("log processor").set("spark.executor.memory", "2g"))

sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint")


In [3]:
# create DStream from text file
# Note: the spark streaming checks for any updates to this directory.
# So first, start this program, and then copy the log file access_log.log to directory location
log_data = ssc.textFileStream('logs')
access_log_dstream = log_data.map(ApacheAccessLog.parse_from_log_line).filter(lambda parsed_line: parsed_line is not None)
ip_dstream = access_log_dstream.map(lambda parsed_line: (parsed_line.ip, 1)) 
ip_count = ip_dstream.reduceByKey(lambda x,y: x+y)
ip_count.pprint(num = 30)
ip_bytes_dstream = access_log_dstream.map(lambda parsed_line: (parsed_line.ip, parsed_line.content_size))
ip_bytes_sum_dstream = ip_bytes_dstream.reduceByKey(lambda x,y: x+y)
ip_bytes_request_count_dstream = ip_count.join(ip_bytes_sum_dstream)
ip_bytes_request_count_dstream.pprint(num = 30)

In [4]:
####### TODO: use window()to count data over a window##########################
access_logs_window = access_log_dstream.window(windowDuration = 6, slideDuration=4) 
window_counts = access_logs_window.count()
window_counts.pprint()

####### Exercise End ##########################################################

In [5]:
ssc.start() 
# ssc.awaitTermination()

-------------------------------------------
Time: 2018-03-01 18:07:16
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:16
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:18
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:18
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:18
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:20
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:20
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:22
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:22
----------

-------------------------------------------
Time: 2018-03-01 18:07:58
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:07:58
-------------------------------------------



In [6]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2018-03-01 18:08:00
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:08:00
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:08:02
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:08:02
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:08:02
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:08:04
-------------------------------------------

-------------------------------------------
Time: 2018-03-01 18:08:04
-------------------------------------------



## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#discretized-streams-dstreams